In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms
import copy

import numpy as np
import pandas as pd

%matplotlib inline 
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import math

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture

from skimage.filters import sobel
from skimage.color import rgb2gray

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

In [ ]:
x = np.load("data/x_train_b.npy")
y = np.load("data/y_train_b.npy")
print(x.shape)
print(y.shape)

In [ ]:
import matplotlib.pyplot as plt

percentages= [np.count_nonzero(y==0),
              np.count_nonzero(y==1),
              np.count_nonzero(y==2)]
              
fig1, ax1 = plt.subplots()
ax1.pie(percentages,labels=["1","2","3"], autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  

plt.show()

In [ ]:
def moving_average(arr, n=3):
  padded_arr = np.concatenate((np.flip(arr[:n]), arr, np.flip(arr[-n:])))
  return np.convolve(padded_arr, np.ones(2 * n + 1)/(2 * n + 1), mode='valid')

def auto_crop(image, target_width=750, target_height=130, xbuffer=15, ybuffer=10):
  edges = sobel(rgb2gray(image))
  columns = np.sum(edges, axis=0)
  smoothed_columns = np.array(moving_average(columns, 2 * xbuffer + 1))

  left = 0
  right = len(smoothed_columns) - 1
  while right - left + 1 > target_width:
    if smoothed_columns[left] < smoothed_columns[right]:
      left += 1
    else:
      right -= 1
  image = image[:, left:right+1, :]    
  edges = edges[:, left:right+1]
  # plt.plot(columns)
  # plt.plot(smoothed_columns)
  # plt.axvline(left, color='r')
  # plt.axvline(right, color='r')
  # plt.show()
  
  rows = np.sum(edges, axis=1)
  smoothed_rows = np.array(moving_average(rows, 2 * ybuffer + 1))
  top = 0
  bottom = len(smoothed_rows) - 1
  while bottom - top + 1 > target_height:
    if smoothed_rows[top] < smoothed_rows[bottom]:
      top += 1
    else:
      bottom -= 1
  # plt.plot(rows)
  # plt.plot(smoothed_rows)
  # plt.axvline(top, color='r')
  # plt.axvline(bottom, color='r')
  # plt.show()
  image = image[top:bottom+1, :, :]
  return image

In [ ]:
# test image cropping
# index = random.randrange(len(x))
# print(f'index = {index}')
# f, axarr = plt.subplots(2, 4, figsize = (24, 3))
# for i in range(4):
#   axarr[1, i].imshow(auto_crop(x[index][i]))
#   axarr[0, i].imshow(x[index][i])
# plt.show()

In [ ]:
# crop all images
cropped_x = [[-1] * 4] * len(x)
for i in range(len(x)):
  for j in range(4):
    cropped_x[i][j] = auto_crop(x[i][j])
  cropped_x[i] = np.stack(cropped_x[i])
cropped_x = np.stack(cropped_x)
print(x.shape)

for i in np.random.choice(range(len(x)), size=2, replace=False):
  f, axarr = plt.subplots(2, 4, figsize = (24, 3))
  for j in range(4):
    axarr[0, j].imshow(x[i][j])
    axarr[1, j].imshow(cropped_x[i][j])
  plt.show()
  
x = cropped_x

In [ ]:
tensor_x = torch.Tensor(x) 
tensor_y = torch.Tensor(y).long()

tensor_x = torch.swapaxes(tensor_x,2,4)
tensor_x = torch.swapaxes(tensor_x,3,4)

x_train,x_test,y_train,y_test = train_test_split(tensor_x,tensor_y,test_size=0.1)

print("x train shape: ", x_train.shape,"x test shape: ",x_test.shape)

train_ds = TensorDataset(x_train,nn.functional.one_hot(y_train,3)) 
test_ds = TensorDataset(x_test,nn.functional.one_hot(y_test,3)) 

train_dl = DataLoader(train_ds,2,shuffle = True)
test_dl = DataLoader(test_ds,2,drop_last = True)
del x,y

In [ ]:
# fig, (ax1, ax2) = plt.subplots(1, 2)
# fig.suptitle('subplots')

# percentages1 = [np.count_nonzero(y==0),
#               np.count_nonzero(y==1),
#               np.count_nonzero(y==2)]

# percentages1 = [np.count_nonzero(y==0),
#               np.count_nonzero(y==1),
#               np.count_nonzero(y==2)]
              

# ax1.pie(percentages,labels=["1","2","3"], autopct='%1.1f%%',
#         shadow=True, startangle=90)
# ax2.pie(percentages,labels=["1","2","3"], autopct='%1.1f%%',
#         shadow=True, startangle=90)



              
# fig1, ax1 = plt.subplots()
# ax1.pie(percentages,labels=["1","2","3"], autopct='%1.1f%%',
#         shadow=True, startangle=90)
# ax1.axis('equal')  

# plt.show()

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
#print(list(model.parameters())[0][0][0])
model = nn.Sequential(*list(model.children())[:-1])
# list(model.children())
#print(list(model.parameters())[0][0][0])

In [ ]:
def reluToInplaceFalse(model):
  for name, child in model.named_children():
    if isinstance(child, nn.ReLU):
      setattr(child, 'inplace', False)
    else:
      reluToInplaceFalse(child)

In [ ]:
from torchvision.transforms.transforms import RandomRotation

class Classifier(torch.nn.Module):

  def __init__(self, backbone='resnet', multi_backbone = True, device ="cuda:0",dropout_rate = 0.2, do_augmentation = False):
    super().__init__()
    self.multi_backbone = multi_backbone

    if backbone == "vgg19":
      backbone = torchvision.models.vgg19(pretrained=True)
      self.out_channels = 25088
      
    elif backbone == "resnet18":
      backbone = torchvision.models.resnet18(pretrained=True)
      self.out_channels = 512

    elif backbone == "resnet50":
      backbone = torchvision.models.resnet50(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b1":
      backbone = torchvision.models.efficientnet_b1(pretrained=True)
      self.out_channels = 1280

    elif backbone == "Efficientnet b3":
      backbone = torchvision.models.efficientnet_b3(pretrained=True)
      self.out_channels = 1536

    elif backbone == "Efficientnet b5":
      backbone = torchvision.models.efficientnet_b5(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b7":
      backbone = torchvision.models.efficientnet_b7(pretrained=True)
      self.out_channels = 2560
      
    # Disabling inplace ReLu becasuse GradCam doesn't work it enabled
    reluToInplaceFalse(backbone)
     
    modules = list(backbone.children())[:-1]
    self.do_augmentation = do_augmentation

    if self.do_augmentation:
      self.augmentation = nn.Sequential(transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomPerspective(0.2),
                                        RandomRotation(20),
                                        transforms.RandomAutocontrast())

    if self.multi_backbone:
      self.backbone1 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone2 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone3 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone4 = nn.Sequential(*copy.deepcopy(modules)).to(device)
    else:
      self.backbone =  nn.Sequential(*modules).to(device)


     

    self.fc1 = nn.Sequential(nn.Dropout(dropout_rate),
                              nn.Linear(self.out_channels, 128),
                              nn.ReLU(),
                              nn.Dropout(dropout_rate)) #TODO: Experiment with BN and Dropout

    self.fc = nn.Sequential(nn.Linear(512, 3))                  #TODO: L2 Regularization
     
  def forward(self, x, is_training = True):
    if self.do_augmentation and is_training:
      imgs = [self.augmentation(x[:,i]) for i in range(4)] #list of 4 images
    else:
      imgs = [x[:,i] for i in range(4)] #list of 4 images

    if self.multi_backbone:
      encodings = [self.fc1(torch.flatten(self.backbone1(imgs[0]),1)),
                   self.fc1(torch.flatten(self.backbone2(imgs[1]),1)),
                   self.fc1(torch.flatten(self.backbone3(imgs[2]),1)),
                   self.fc1(torch.flatten(self.backbone4(imgs[3]),1))]
    else:
      encodings = [self.fc1(self.backbone(img).squeeze()) for img in imgs]

    return self.fc(torch.cat(encodings,1))


In [ ]:
def freeze_layers(model,layers = 5):
  if model.multi_backbone:
    for i,param in enumerate(model.backbone1.parameters()):
      if i < layers:
        param.requires_grad = False

    for i,param in enumerate(model.backbone2.parameters()):
      if i < layers:
        param.requires_grad = False

    for i,param in enumerate(model.backbone3.parameters()):
      if i < layers:
        param.requires_grad = False

    for i,param in enumerate(model.backbone4.parameters()):
      if i < layers:
        param.requires_grad = False
  else:
     for i,param in enumerate(model.backbone.parameters()):
      if i < layers:
        param.requires_grad = False



In [ ]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def train_model(model, train_dl, test_dl, optimizer, evaluation_steps = 20, epochs =1, savefile='model.pt'):
  writer = SummaryWriter()

  EPOCHS = epochs
  DATA_SIZE =  {"train":len(train_dl.dataset),"eval":len(test_dl.dataset)}

  best_eval_acc = 0
  best_eval_acc_step = 0
  
  best_eval_loss = float('inf')
  best_eval_loss_step = 0

  final_eval_loss = 0
  final_eval_acc = 0

  final_train_loss = 0
  final_train_acc = 0




  exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
  criterion = nn.BCEWithLogitsLoss()

  step =0

  for epoch in range(EPOCHS):
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in train_dl:
          optimizer.zero_grad()
          step += 1
          model.train()
          inputs = inputs.to(device)
          labels = labels.to(device)
          outputs = model(inputs)

          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels.squeeze(1).float())
          loss.backward()
          optimizer.step()

          # statistics
          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == torch.argmax(labels,2).squeeze().data)
          # if phase == 'train':
          #     scheduler.step()
          if step % evaluation_steps == 0:
            test_running_loss = 0.0
            test_running_corrects = 0
            model.eval()
            with torch.no_grad():
              for inputs, labels in test_dl:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs,is_training=False)
                loss = criterion(outputs, labels.squeeze(1).float())
                _, preds = torch.max(outputs, 1)


                test_running_loss += loss.item() * inputs.size(0)
                test_running_corrects += torch.sum(preds == torch.argmax(labels,2).squeeze().data)

            eval_loss = test_running_loss / DATA_SIZE["eval"]
            eval_acc = test_running_corrects.double() / DATA_SIZE["eval"]

            writer.add_scalar('Loss/'+"eval", eval_loss, step)
            writer.add_scalar('Accuracy/'+"eval", eval_acc, step)

            final_eval_loss = eval_loss
            final_eval_acc = eval_acc

            if eval_loss < best_eval_loss:
              best_eval_loss = eval_loss
              best_eval_loss_step = step
              torch.save(model, savefile)

            if eval_acc > best_eval_acc:
              best_eval_acc = eval_acc
              best_eval_acc_step = step

            # print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            # "Eval", eval_loss, eval_acc))



        epoch_loss = running_loss / DATA_SIZE["train"]
        epoch_acc = running_corrects.double() / DATA_SIZE["train"]


        final_train_loss = epoch_loss
        final_train_acc = epoch_acc

        # print('Epoch {}/{} '.format(epoch+1, EPOCHS ),'{} Loss: {:.4f} Acc: {:.4f}'.format(
        #       "Train", epoch_loss, epoch_acc))
          
        writer.add_scalar('Loss/'+"Train", epoch_loss, step)
        writer.add_scalar('Accuracy/'+"Train", epoch_acc, step)

  # print('Best val Acc: {:4f}\n\n'.format(best_eval_acc))

  return {  "Best Eval Acc": best_eval_acc.cpu().item(),
            "Best Eval Loss": best_eval_loss,
            "Best Eval Step": best_eval_loss_step,
            "Final Train Acc": final_train_acc.cpu().item(),
            "Final Eval Acc": final_eval_acc.cpu().item(),
            "Final Train Loss": final_train_loss,
            "Final Eval Loss": final_eval_loss,
        }
# load best model weights
# model.load_state_dict(best_model_wts)


In [ ]:

Sweep_Summary = {
    "Backbone" : [],
    "Optim" : [],
    "LR" : [],
    "Scheduler" : [],
    "Epochs" : [],
    "Batch Size" : [],
    "Dropout": [],
    "Augmentation": [],
    "Weight Decay": [],
    "Freeze Layers": [],
    "Best Eval Loss": [],
    "Best Eval Step": [],
    "Best Eval Acc": [],
    "Final Train Acc": [],
    "Final Eval Acc": [],
    "Final Train Loss": [],
    "Final Eval Loss": [],

}

BACKBONE = "resnet18"
OPTIM = "Adam"
LR = 2e-5
SCHEDULER = "None"
EPOCHS = 40
BATCHSIZE = 2
DROPOUT = [0,0.2,0.5]
AUGMENTATION = "None"
WEIGHT_DECAY = [0,1e-3,1e-5]
FREEZE = [155,150,140,130,110]

DROPOUT = [0.5]
WEIGHT_DECAY = [0.01]
FREEZE = [50]


for weight_decay in WEIGHT_DECAY:
  for freeze in FREEZE:
    for dropout_rate in DROPOUT:
      print("Weight Decay {} ".format(weight_decay))
      print("Freeze       {} ".format(freeze))
      print("Dropout Rate {} \n------------------------------------------------------".format(dropout_rate))
      model = Classifier(dropout_rate=dropout_rate,backbone=BACKBONE,multi_backbone=True).to(device)
      freeze_layers(model,freeze)
      optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=weight_decay)
      summary = train_model(model,train_dl,test_dl,optimizer,epochs=EPOCHS,savefile='model.pt')

      Sweep_Summary["Backbone"].append(BACKBONE)
      Sweep_Summary["Optim"].append(OPTIM)
      Sweep_Summary["LR"].append(LR)
      Sweep_Summary["Scheduler"].append(SCHEDULER)      
      Sweep_Summary["Epochs"].append(EPOCHS)
      Sweep_Summary["Batch Size"].append(BATCHSIZE)
      Sweep_Summary["Dropout"].append(dropout_rate)
      Sweep_Summary["Augmentation"].append(AUGMENTATION)
      Sweep_Summary["Weight Decay"].append(weight_decay)
      Sweep_Summary["Freeze Layers"].append(freeze)
      for key,value in summary.items():
        Sweep_Summary[key].append(value)

In [ ]:
# Test model loading
saved_model = torch.load('model.pt')
saved_model.eval()
test_running_loss = 0.0
all_preds = []
all_labels = []
criterion = nn.BCEWithLogitsLoss()
dl = test_dl
with torch.no_grad():
  for inputs, labels in dl:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = saved_model(inputs,is_training=False)
    loss = criterion(outputs, labels.squeeze(1).float())
    _, preds = torch.max(outputs, 1)
    all_preds.extend(preds.cpu().tolist())
    all_labels.extend([label[0] for label in labels.cpu().tolist()])


    test_running_loss += loss.item() * inputs.size(0)

all_preds = np.array(all_preds)
all_labels = np.argmax(all_labels, axis = 1)

eval_loss = test_running_loss / len(dl.dataset)
eval_acc = accuracy_score(all_labels, all_preds)
print('{} Loss: {:.4f} Acc: {:.4f}'.format("Eval", eval_loss, eval_acc))
print(f'Pred: {all_preds}')
print(f'True: {all_labels}')
print('Confusion Matrix:')
print(confusion_matrix(all_labels, all_preds))

In [ ]:
df = pd.DataFrame(Sweep_Summary)
df.to_csv("Fish_Classifier_Sweep_Results_2.0.csv")
!cp "/content/Fish_Classifier_Sweep_Results_2.0.csv" "/content/drive/MyDrive"

In [ ]:
from torchsummary import summary
torch.cuda.empty_cache()
model = Classifier(dropout_rate=dropout_rate,backbone=BACKBONE,multi_backbone=True).to(device)
summary(model, (4, 3, 200, 1000))

In [ ]:
i =0
for _ in model.backbone1.parameters():
  i += 1
i

In [ ]:
df

In [ ]:
!pip install wandb
!wandb log. in
#6533b3e866937401e2fae54e38119aba0c36d8e4

In [ ]:
!tensorboard dev upload --logdir runs

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
LOG_DIR = './runs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
!./ngrok authtoken 24nct5MrkkabWHKHo1qlKir5TJo_6TYt5jNTyni2WRqKegBYy